In [1]:
from __future__ import print_function
import library_py2 as lib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.collocations import *
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import argparse

Using TensorFlow backend.
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:871: DeprecationWarning: builtin type EagerTensor has no __module__ attribute
  EagerTensor = c_api.TFE_Py_InitEagerTensor(_EagerTensorBase)
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/keras/_impl/keras/backend.py:4422: ResourceWarning: unclosed file <_io.TextIOWrapper name='/Users/flatir

In [ ]:
# code for FINAL CLEANING, i.e. all the extra nobs that p'd me off so much 
df = pd.read_pickle('lstm_df.pkl')

df.drop(columns=['plausible_answers_lstm'], inplace=True)

df = lib.preprocessing(df, columns_list=['context_lstm', 'question_lstm', 'answers_lstm'], 
                      lower_all=True, pos_tagging=True, lemm=True, suffix='_1')

df['context_lstm'] = df.context_lstm__1.apply(lib.extract)
df['question_lstm'] = df.question_lstm__1.apply(lib.extract)
df['answers_lstm'] = df.answers_lstm__1.apply(lib.extract)

df['text'] = df['context_lstm'] + ' ' + df['question_lstm'] 

tdf = df.drop(columns=['context_lstm', 'question_lstm', 'context_lstm__1', 'question_lstm__1','answers_lstm__1'])

tdf['text_len'] = (tdf['text'].apply(lambda x: len(x.split(' '))))
tdf['answers_lstm_len'] = (tdf['answers_lstm'].apply(lambda x: len(x.split(' '))))

tdf['fulltext']= tdf['text'] + ' ' + tdf['answers_lstm']

tdf.fulltext.apply(lambda x: normalize('NFKD', x).encode('ascii','ignore'))

tdf.answers_lstm = tdf.answers_lstm.apply(lambda x: x.replace('\n', ' '))
tdf.text = tdf.text.apply(lambda x: x.replace('\n', ' '))
tdf.fulltext = tdf.fulltext.apply(lambda x: x.replace('\n', ' '))

ndf = tdf.loc[tdf.answers_lstm_len <=3]

ndf.to_pickle('lstm_1.pkl')
tdf.to_pickle('lstm_1_long_answers.pkl')


In [3]:
!pwd

/Users/flatironschool/Desktop/Io/NLGforedtech


In [12]:
data_path = "/home/jupyter/NLGforedtech/"

In [4]:
df = pd.read_pickle('lstm_df.pkl')

In [5]:
df.head()

,context_lstm,question_lstm,answers_lstm,plausible_answers_lstm
0,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,when do beyonce start become popular,in the late 1990s,
1,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,what area do beyonce compete in when she be gr...,singing and dancing,
2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,when do beyonce leave destinys child and becom...,2003,
3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,in what city and state do beyonce grow up,houston texas,
4,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,in which decade do beyonce become famous,late 1990s,


In [6]:
df.drop(columns=['plausible_answers_lstm'], inplace=True)

In [7]:
vocabulary = lib.get_total_vocab(df, columns=['context_lstm', 'question_lstm'])

In [8]:
len(vocabulary)

92265

In [9]:
vocabulary['table']

43163

In [18]:
vocabulary.keys()

dict_keys(['', 'martti', 'cahit', 'brainstorming', 'castellum', 'thirdrichest', 'río', 'beixiang', 'mnotor', 'isisdemeter', 'forgotten', 'us599', 'p91', 'torchs', 'byung', 'tsuibushi', 'heterotrophic', 'brazils', '105946', 'bengels', 'megacity', 'sharia', 'sclerotized', '1345', 'quarterwave', 'mortain', '1697', 'anatoly', 'deval', 'crisiscenter', 'skirt', 'achn', 'musō', 'hellinistic', 'kamakura', 'surgeonfish', 'znso\n4', '1222', 'westin', 'possessed', 'kray', 'reciever', 'mattel', 'threestep', 'amc', 'schutzs', 'toc1', 'following', 'job', 'tripoli', 'exarate', 'beachhead', 'adultlike', 'franich', 'welltodo', 'deathless', 'gameday', '674', 'mi2', 'branch', 'laocoön', '300th', 'domesticated', 'primordialist', '10–15', 'tahia', '1047', 'namesake', 'glencoe', 'jugendstil', 'bestclarification', '69006400', 'unevenly', 'informal', 'kannauj', 'valveless', 'decompresses', 'trubetzkoys', 'swifts', 'timewhen', 'physicist', 'pastor', 'spiralling', 'condsidered', 'negotiation', 'nonmexican', 'mm

In [10]:
df.context_lstm.nunique()

18875

In [20]:
wod = lib.file_to_word_ids(vocabulary, df[:10], columns=['context_lstm', 'question_lstm'])

In [21]:
wod

[0,
 62229,
 6392,
 6779,
 7347,
 8248,
 11053,
 12861,
 13048,
 13367,
 13437,
 13634,
 14209,
 14909,
 15323,
 15567,
 16904,
 12163,
 18929,
 19885,
 21831,
 22488,
 23168,
 7417,
 24647,
 32033,
 25638,
 25733,
 26158,
 28634,
 30329,
 30557,
 30749,
 31179,
 31537,
 32711,
 33265,
 34253,
 34406,
 35393,
 24810,
 52309,
 38546,
 41588,
 38839,
 39724,
 42263,
 53379,
 37921,
 45337,
 46375,
 46588,
 47317,
 7905,
 48535,
 49343,
 54749,
 50695,
 51027,
 51648,
 64399,
 53235,
 53802,
 54649,
 58399,
 61219,
 61237,
 10111,
 61889,
 64347,
 65433,
 65520,
 67248,
 67948,
 68579,
 17635,
 69916,
 70070,
 70736,
 42632,
 74279,
 74444,
 75089,
 75572,
 75687,
 76081,
 77901,
 79364,
 79427,
 79617,
 79865,
 79948,
 28956,
 82568,
 13927,
 83636,
 44780,
 84694,
 85682,
 87314,
 87371,
 87399,
 30148,
 45618,
 89296,
 91309,
 46045]

In [31]:
# fn where we padd the input question
df['question_lstm_len'] = (df['question_lstm'].apply(lambda x: len(x.split(' '))))
df['answers_lstm_len'] = (df['answers_lstm'].apply(lambda x: len(x.split(' '))))


In [32]:
df.question_lstm[0]

'when do beyonce start become popular '

In [33]:
df.question_lstm_len.max()

41

In [44]:
df.answers_lstm_len.max()

44

In [36]:
df.loc[df.answers_lstm_len==44]

,context_lstm,question_lstm,answers_lstm,question_lstm_len,answers_lstm_len
3201,an article in science suggest that the constru...,what be conclude about the construction,that the sudden shift of a huge quantity of wa...,7,44


In [34]:
df.head()

,context_lstm,question_lstm,answers_lstm,question_lstm_len,answers_lstm_len
0,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,when do beyonce start become popular,in the late 1990s,7,5
1,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,what area do beyonce compete in when she be gr...,singing and dancing,12,4
2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,when do beyonce leave destinys child and becom...,2003,12,2
3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,in what city and state do beyonce grow up,houston texas,10,3
4,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,in which decade do beyonce become famous,late 1990s,8,3


In [62]:
len(df.loc[df.answers_lstm_len<=3])

52129

In [56]:
df.question_lstm.apply(lambda x: pad_in(x, df.question_lstm_len.max()))

0                 when do beyonce start become popular     
1         what area do beyonce compete in when she be gr...
2         when do beyonce leave destinys child and becom...
3                in what city and state do beyonce grow up 
4                 in which decade do beyonce become famous 
                                ...                        
130046    in what us state do kathmandu first establish ...
130047            what be yangon previously know as        
130048    with what belorussian city do kathmandu have a...
130049    in what year do kathmandu create its initial i...
130050            what be kmc an initialism of             
Name: question_lstm, Length: 86821, dtype: object

In [59]:
len(df.question_lstm[0])

37

In [16]:
df.question_lstm_len.max()

86821

In [73]:
df['context&question'] = df['context_lstm'] + ' ' + df['question_lstm'] 
df['fulltext'] = df['context&question'] + ' ' + df['answers_lstm']

In [74]:
    # the training df for the 
context = df['context_lstm'].drop_duplicates()


In [75]:
len(context)

18875

In [76]:
tdf = df.loc[df.answers_lstm_len <=3]

In [80]:
tdf.head()

,context_lstm,question_lstm,answers_lstm,question_lstm_len,answers_lstm_len,fulltext,context&question
2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,when do beyonce leave destinys child and becom...,2003,12,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,in what city and state do beyonce grow up,houston texas,10,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
4,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,in which decade do beyonce become famous,late 1990s,8,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
5,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,in what rb group be she the lead singer,destiny child,10,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
7,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,who manage the destiny child group,mathew knowles,7,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...


In [81]:
def load_data():
    # 
    train_df = tdf.iloc[:round(0.6*len(tdf))]
    valid_df = tdf.iloc[round(0.6*len(tdf)):round(0.8*len(tdf))]
    test_df = tdf.iloc[round(0.8*len(tdf)):]
    

    # build the complete vocabulary, then convert text data to dict of integer-word pairs
    vocabulary = lib.get_total_vocab(tdf, columns=['context&question'])
    train_data = lib.file_to_word_ids(vocabulary, train_df, columns=['context&question'])
    valid_data = lib.file_to_word_ids(vocabulary, valid_df, columns=['context&question'])
    test_data = lib.file_to_word_ids(vocabulary, test_df, columns=['context&question'])
    vocabulary_size = len(vocabulary)
    reversed_dictionary = dict(zip(vocabulary.values(), vocabulary.keys()))

    print(train_data[:5])
    print(vocabulary)
    print(vocabulary_size)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_data, valid_data, test_data, vocabulary, reversed_dictionary, vocabulary_size



In [82]:
vocabulary['table']

43163

In [83]:
train_data, valid_data, test_data, vocabulary, reversed_dictionary, vocabulary_size = load_data()

[33238, 22003, 30403, 57199, 83462]
{'': 0, 'tyler': 1, 'buckcherrys': 2, 'eia': 3, 'noni': 4, 'culte': 5, '5814': 6, 'xhtjb': 7, 'vitruviuss': 8, 'literature”': 9, 'printer': 10, 'semantically': 11, 'solves': 12, 'loango': 13, 'halfhour': 14, 'delhi': 15, 'lewinter': 16, 'impedance': 17, 'monophthongs': 18, 'barolo': 19, 'hfpa': 20, 'scp': 21, 'c1000': 22, 'legitimise': 23, 'presupposition': 24, 'flip': 25, '29000': 26, 'eightyone': 27, 'semitones': 28, 'cation': 29, 'parc': 30, 'militarially': 31, 'balus': 32, 'invite': 33, 'forehead': 34, 'y': 35, 'gouraud': 36, 'recalibrating': 37, 'yingxiu': 38, 'subschools': 39, 'divest': 40, '312000': 41, 'wiesen': 42, 'isomer': 43, 'abunaser': 44, 'ethiopias': 45, 'xfiles': 46, 'endosymbiont': 47, 'crytzer': 48, 'montjoy': 49, 'cha': 50, 'senad': 51, 'algarves': 52, '11396': 53, 'koszycka': 54, 'tafts': 55, 'sohn': 56, 'exyuan': 57, 'whalebone': 58, 'networkqriocity': 59, 'khattak': 60, '1512': 61, 'populaions': 62, 'penguin': 63, 'biotechnolog

In [84]:
reversed_dictionary[9927]

'outreach'

In [50]:
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical


class KerasBatchGenerator(object):
    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        self.current_idx = 0
        self.skip_step = skip_step
        self.vocabulary_size = len(vocabulary)
        
        
    #     num_steps = nr of words that will be fed into the input layer; i.e. the nr of words the model will use to predict the next word
    #     skip_steps = how far to move the window after the prediction is made

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps)) # input nn nodes, same dimensions as batchsize x nr of words in one "window"
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary_size)) # output nodes
        
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data): # once the ticker goes over the length of data, reset it
                    self.current_idx = 0
                x[i,:] = self.data[self.current_idx:self.current_idx + self.num_steps]
                # input is the segment of data from current index to current index + step-increment
                temp_y = self.data[(self.current_idx + 1) : (self.num_steps + 1) ]
                # converts all the output y into a 1-hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary_size)
                self.current_idx += self.skip_step
                
            yield x, y
            
        # x dimensions: nr of samples we specify in the batch first, number of w to base predictions on 
        # y has 3 dims: batchsize, nr time steps and also size of vocabulary
        from keras.callbacks import ModelCheckpoint


In [51]:
num_steps = 30
batch_size = 20


In [52]:
        
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)

In [53]:
hidden_size = 500
use_dropout=True
num_epochs=3

In [54]:
model = Sequential()
model.add(Embedding(vocabulary_size, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary_size)))
model.add(Activation('softmax'))

optimizer=Adam()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())




/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3


ValueError: could not broadcast input array from shape (0,10643) into shape (30,10643)

In [49]:

if args.run_opt ==1:
    model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,)
    #                         validation_data=valid_data_generator.generate(),
    #                         validation_steps=len(valid_data)//(batch_size))
    model.save(data_path + "/final_model.hdf5")
elif args.run_opt ==2:
    model = load_model(data_path + "/mo")


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 500)           5321500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 500)           2002000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 500)           2002000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 500)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 10643)         5332143   
_________________________________________________________________
activation_1 (Activation)    (None, 30, 10643)         0         
Total params: 14,657,643
Trainable params: 14,657,643
Non-trainable params: 0
__________________________________________